In [1]:
import pandas as pd
import numpy as np
import random
import warnings
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#1.read data
path = "D:/python/contest/picture/fer2013/"
file = "fer2013.csv"
data = pd.read_csv(path+file)
data['pixels'] = data['pixels'].apply(lambda x: np.array([int(pixel) for pixel in x.split(' ')]))

In [56]:
#2.split data into sets
train_set   = data.loc[data['Usage'] == 'Training',['emotion','pixels']].reset_index(drop=True)
public_set  = data.loc[data['Usage'] == 'PublicTest',['emotion','pixels']].reset_index(drop=True)
private_set = data.loc[data['Usage'] == 'PrivateTest',['emotion','pixels']].reset_index(drop=True)
# use train_set to train
# use public_set to evaluate

In [50]:
#3.make batch generator
#  using generator to save the memory
def sample_generator(df,batch_size):
    while True:
        index = np.arange(df.shape[0]) # get index of all picture
        random.shuffle(index)          # shuffle the index
        for i in range(0,int(np.ceil(df.shape[0]/batch_size))):
            id1 = i*batch_size
            id2 = min((i+1)*batch_size,len(index))
            batch_id = index[id1:id2]
            batch_X  = df.loc[batch_id,'pixels'].tolist()
            batch_Y  = df.loc[batch_id,'emotion'].tolist()
            #make the batch data to array format            
            batch_X  = [x.reshape(1,48,48,1) for x in batch_X]
            #regularize the data
            batch_X  = [x/128-1 for x in batch_X]
            batch_X  = np.concatenate(batch_X,axis=0)
            batch_Y  = to_categorical(batch_Y,num_classes=7)
            yield batch_X , batch_Y 

In [47]:
#4.build model by keras
model = Sequential()
#add model layers
model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(64, kernel_size=4, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
def train_test_split(df,split):
    index = np.arange(df.shape[0]) # get index of all picture
    random.shuffle(index)
    split_id = round(df.shape[0]*split)
    train_index,test_index = index[:split_id],index[split_id:]
    train_df = df.loc[train_index].copy(deep=True).reset_index(drop=True)
    test_df  = df.loc[test_index].copy(deep=True).reset_index(drop=True)
    return train_df,test_df

In [51]:
#5.train
batch_size = 128
# train_test_split
train_train,train_test = train_test_split(train_set,0.7)
model.fit_generator(sample_generator(train_train,batch_size),
                    steps_per_epoch = np.ceil(train_train.shape[0]/batch_size),
                    validation_data = sample_generator(train_test,batch_size),
                    validation_steps= np.ceil(train_test.shape[0]/batch_size),
                    epochs = 10,verbose=2)

Epoch 1/10
 - 4s - loss: 1.7588 - acc: 0.2759 - val_loss: 1.6225 - val_acc: 0.3595
Epoch 2/10
 - 3s - loss: 1.5061 - acc: 0.4179 - val_loss: 1.4659 - val_acc: 0.4316
Epoch 3/10
 - 3s - loss: 1.3703 - acc: 0.4785 - val_loss: 1.3650 - val_acc: 0.4781
Epoch 4/10
 - 3s - loss: 1.2796 - acc: 0.5158 - val_loss: 1.2788 - val_acc: 0.5206
Epoch 5/10
 - 3s - loss: 1.2040 - acc: 0.5483 - val_loss: 1.2499 - val_acc: 0.5283
Epoch 6/10
 - 3s - loss: 1.1516 - acc: 0.5667 - val_loss: 1.2442 - val_acc: 0.5315
Epoch 7/10
 - 3s - loss: 1.1031 - acc: 0.5858 - val_loss: 1.2224 - val_acc: 0.5416
Epoch 8/10
 - 3s - loss: 1.0398 - acc: 0.6118 - val_loss: 1.2244 - val_acc: 0.5347
Epoch 9/10
 - 3s - loss: 0.9898 - acc: 0.6348 - val_loss: 1.2227 - val_acc: 0.5408
Epoch 10/10
 - 3s - loss: 0.9425 - acc: 0.6548 - val_loss: 1.2516 - val_acc: 0.5391


In [60]:
#6.evaluate
result = model.evaluate_generator(sample_generator(public_set,batch_size),
                         steps=np.ceil(public_set.shape[0]/batch_size))
print(f"public accuracy:{result[1]}")

public accuracy:0.5397046531399301
